# Проблема поиска собственных значений

In [1]:
import numpy as np

In [2]:
n = 4

In [3]:
# n random eigenvalues
rnd_eigen_values = np.random.rand(n)[::-1] * 10
rnd_eigen_values

array([1.2119286 , 9.88739975, 4.83278368, 4.05656272])

In [4]:
# build diagonal matrix
L = np.diag(rnd_eigen_values)
L

array([[1.2119286 , 0.        , 0.        , 0.        ],
       [0.        , 9.88739975, 0.        , 0.        ],
       [0.        , 0.        , 4.83278368, 0.        ],
       [0.        , 0.        , 0.        , 4.05656272]])

In [5]:
# random matrix C
C = np.random.rand(n, n) * 15
assert np.linalg.det(C) != 0, 'det(A) = 0'
C

array([[ 0.15973155, 11.02947827,  1.48460893,  7.39103385],
       [ 4.46226731,  4.27491198,  7.86565733,  2.45898502],
       [14.9327468 ,  8.57249691, 10.4172719 ,  6.7982045 ],
       [ 0.13564799, 11.39431795,  7.06111135,  2.78119386]])

In [6]:
A = np.linalg.inv(C) @ L @ C
A

array([[  0.5767198 ,  -1.81275647,  -7.43711866,  -0.66326198],
       [ -5.3925659 ,   0.13678149, -10.20393049,  -2.00228634],
       [  6.07425452,   6.20349107,  15.75493969,   3.45502159],
       [  6.84082359,   0.39752326,  12.46663048,   3.52023377]])

In [7]:
np.linalg.eigvals(A)

array([9.88739975, 1.2119286 , 4.83278368, 4.05656272])

In [8]:
max(np.linalg.eigvals(A))

9.887399753806967

## Степенной метод

In [9]:
def degree_method(A, n, eps=1e-6, delta=0.01, val='max'):
    y0 = np.random.rand(n)
    z = y0 / np.linalg.norm(y0)
    l_cur, l_prev = None, None
    iters = 0
    
    if val == 'min':
        A = np.linalg.inv(A)        
    
    while iters < 2 or abs(l_prev - l_cur) >= eps:
        y = A @ z
            
        l_prev = l_cur
        
        l = []
        for i in range(len(z)):
            if abs(z[i]) > delta:
                if val == 'min':
                    l.append(z[i] / y[i])
                else:
                    l.append(y[i] / z[i])
        l_cur = np.mean(l)
            
        z = y / np.linalg.norm(y)
        iters += 1
    
    print('Кол-во итераций:', iters)
    
    return l_cur, z

In [10]:
l, x = degree_method(A, n, val='min')
l, x

Кол-во итераций: 14


(1.2119283394689446,
 array([-0.29371367, -0.22103432, -0.00397582,  0.92997866]))

## Обратный степенной метод

In [11]:
def reverse_degree_method(A, n: int, k: int, eps=1e-6, delta=0.01, verbose=False):
    
    def get_for_s(A, n):
        s_prev = None
        s = float(np.random.rand(1) * 10)
        y0 = np.random.rand(n)
        z = y0 / np.linalg.norm(y0)
        iters = 0

        while iters < 1000 and (iters < 2 or abs(s - s_prev) >= eps):
            y = np.linalg.inv((A - s*np.eye(n))) @ z

            l = []
            for i in range(len(z)):
                if abs(z[i]) > delta:
                    l.append(z[i] / y[i])
            s_prev = s
            s += np.mean(l)

            z = y / np.linalg.norm(y)
            iters += 1
        
        if verbose:
            print(f'Iters for s={s}: iters={iters}')

        return s, z
    
    es, ex = [], []
    for _ in range(k):
        s0 = np.random.rand(1)
        s, x = get_for_s(A, n)
        es.append(s)
        ex.append(x)
    
    return es, ex

In [12]:
l, x = reverse_degree_method(A, n, 30, eps=1e-6, verbose=False)

l = list(set([round(s, 5) for s in l]))

l

[1.21193, 4.83278, 4.05656, 9.8874]